<a href="https://colab.research.google.com/github/kashyapkakadiya/spotify-song-analytics-etl-pipeline/blob/main/Spotify_Song_Analytics_ETL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Extract

In [8]:
import pandas as pd

df = pd.read_csv("/content/Most Streamed Spotify Songs 2024.csv", encoding='ISO-8859-1')

print(df.head())

                        Track                    Album Name          Artist  \
0         MILLION DOLLAR BABY  Million Dollar Baby - Single   Tommy Richman   
1                 Not Like Us                   Not Like Us  Kendrick Lamar   
2  i like the way you kiss me    I like the way you kiss me         Artemas   
3                     Flowers              Flowers - Single     Miley Cyrus   
4                     Houdini                       Houdini          Eminem   

  Release Date          ISRC All Time Rank  Track Score Spotify Streams  \
0    4/26/2024  QM24S2402528             1        725.4     390,470,936   
1     5/4/2024  USUG12400910             2        545.9     323,703,884   
2    3/19/2024  QZJ842400387             3        538.4     601,309,283   
3    1/12/2023  USSM12209777             4        444.9   2,031,280,633   
4    5/31/2024  USUG12403398             5        423.3     107,034,922   

  Spotify Playlist Count Spotify Playlist Reach  ...  SiriusXM Spins  \
0 

#Transformation

In [9]:
import numpy as np

# Remove commas and convert to numeric for stream and playlist columns
numeric_cols = [
    'Spotify Streams',
    'Spotify Playlist Count',
    'Spotify Playlist Reach',
    'SiriusXM Spins',
    'Deezer Playlist Count',
    'Deezer Playlist Reach',
    'Amazon Playlist Count',
    'Pandora Streams',
    'Pandora Track Stations',
    'Soundcloud Streams',
    'Shazam Counts'
]

for col in numeric_cols:
    df[col] = df[col].replace({',': ''}, regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to float/int

# Convert Release Date to datetime
df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')

# Standardize Explicit Track
df['Explicit Track'] = df['Explicit Track'].fillna(0).astype(int)

# Check and handle missing values (optional)
print(df.isnull().sum())   # Print count of nulls in each column

# Fill or drop missing values if needed
df = df.dropna(subset=['Track', 'Artist', 'Spotify Streams'])  # Drop rows missing important fields


Track                            0
Album Name                       0
Artist                           5
Release Date                     0
ISRC                             0
All Time Rank                    0
Track Score                      0
Spotify Streams                113
Spotify Playlist Count          70
Spotify Playlist Reach          72
Spotify Popularity             804
YouTube Views                  308
YouTube Likes                  315
TikTok Posts                  1173
TikTok Likes                   980
TikTok Views                   981
YouTube Playlist Reach        1009
Apple Music Playlist Count     561
AirPlay Spins                  498
SiriusXM Spins                2123
Deezer Playlist Count          921
Deezer Playlist Reach          928
Amazon Playlist Count         1055
Pandora Streams               1106
Pandora Track Stations        1268
Soundcloud Streams            3333
Shazam Counts                  577
TIDAL Popularity              4600
Explicit Track      

#Filling Missing Data

In [10]:
# Drop rows missing key fields
df_clean = df.dropna(subset=['Track', 'Artist', 'Spotify Streams'])

# For (optionally) filling missing numbers elsewhere
numeric_stats = [
    'Spotify Playlist Count', 'Spotify Playlist Reach', 'Spotify Popularity',
    'YouTube Views', 'YouTube Likes', 'TikTok Posts', 'TikTok Likes', 'TikTok Views',
    'YouTube Playlist Reach', 'Apple Music Playlist Count', 'AirPlay Spins',
    'SiriusXM Spins', 'Deezer Playlist Count', 'Deezer Playlist Reach',
    'Amazon Playlist Count', 'Pandora Streams', 'Pandora Track Stations',
    'Soundcloud Streams', 'Shazam Counts', 'TIDAL Popularity'
]

for col in numeric_stats:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(0)  # Fill missing with zero

print(df_clean.info())  # Check that missing values are handled
print(df_clean.head())  # Preview the cleaned data


<class 'pandas.core.frame.DataFrame'>
Index: 4487 entries, 0 to 4599
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Track                       4487 non-null   object        
 1   Album Name                  4487 non-null   object        
 2   Artist                      4487 non-null   object        
 3   Release Date                4487 non-null   datetime64[ns]
 4   ISRC                        4487 non-null   object        
 5   All Time Rank               4487 non-null   object        
 6   Track Score                 4487 non-null   float64       
 7   Spotify Streams             4487 non-null   float64       
 8   Spotify Playlist Count      4487 non-null   float64       
 9   Spotify Playlist Reach      4487 non-null   float64       
 10  Spotify Popularity          4487 non-null   float64       
 11  YouTube Views               4487 non-null   object        
 1

#Load Data into SQLite

In [13]:
import sqlite3

# Create a database connection (file will be created in Colab’s environment)
conn = sqlite3.connect('spotify_songs_2024.db')

# Store DataFrame in a table called "songs"
df_clean.to_sql('songs', conn, index=False, if_exists='replace')

# Test: Query top 5 by Spotify Streams
query = "SELECT Track, Artist, \"Spotify Streams\" FROM songs ORDER BY \"Spotify Streams\" DESC LIMIT 5"
result = pd.read_sql_query(query, conn)
print(result)

               Track         Artist  Spotify Streams
0    Blinding Lights     The Weeknd     4.281469e+09
1    Blinding Lights        xSyborg     4.261328e+09
2       Shape of You     Ed Sheeran     3.909459e+09
3       Shape of You        xSyborg     3.888356e+09
4  Someone You Loved  Lewis Capaldi     3.427499e+09


#Analytics and Querying

###1. Top 10 most streamed songs

In [15]:
query = "SELECT Track, Artist, \"Spotify Streams\" FROM songs ORDER BY \"Spotify Streams\" DESC LIMIT 10"
print(pd.read_sql_query(query, conn))

                                           Track         Artist  \
0                                Blinding Lights     The Weeknd   
1                                Blinding Lights        xSyborg   
2                                   Shape of You     Ed Sheeran   
3                                   Shape of You        xSyborg   
4                              Someone You Loved  Lewis Capaldi   
5  Sunflower - Spider-Man: Into the Spider-Verse    Post Malone   
6                                      As It Was   Harry Styles   
7                                      As It Was   Harry Styles   
8                                        Starboy     The Weeknd   
9                                      One Dance          Drake   

   Spotify Streams  
0     4.281469e+09  
1     4.261328e+09  
2     3.909459e+09  
3     3.888356e+09  
4     3.427499e+09  
5     3.358704e+09  
6     3.301815e+09  
7     3.299082e+09  
8     3.291262e+09  
9     3.192204e+09  


###2. Most popular artists by total streams

In [17]:
query = """
SELECT Artist, SUM("Spotify Streams") AS Total_Streams
FROM songs
GROUP BY Artist
ORDER BY Total_Streams DESC
LIMIT 10
"""
print(pd.read_sql_query(query, conn))

           Artist  Total_Streams
0       Bad Bunny   3.705483e+10
1      The Weeknd   3.694854e+10
2           Drake   3.496216e+10
3    Taylor Swift   3.447077e+10
4     Post Malone   2.613747e+10
5      Ed Sheeran   2.401490e+10
6   Ariana Grande   2.346499e+10
7   MUSIC LAB JPN   2.286669e+10
8  Olivia Rodrigo   1.972922e+10
9          Eminem   1.887888e+10


###3. Trends by Release Year

In [19]:
query = """
SELECT strftime('%Y', "Release Date") AS Year, SUM("Spotify Streams") AS Yearly_Streams
FROM songs
GROUP BY Year
ORDER BY Yearly_Streams DESC
"""
print(pd.read_sql_query(query, conn))

    Year  Yearly_Streams
0   2023    2.252458e+11
1   2022    2.096372e+11
2   2019    1.974695e+11
3   2018    1.909671e+11
4   2017    1.900873e+11
5   2021    1.896088e+11
6   2020    1.815467e+11
7   2016    1.219763e+11
8   2015    1.018258e+11
9   2024    6.939726e+10
10  2014    5.909972e+10
11  2012    5.879390e+10
12  2013    5.645939e+10
13  2011    5.424106e+10
14  2010    2.612400e+10
15  2009    1.618643e+10
16  2008    1.287115e+10
17  2002    9.063840e+09
18  2004    6.163006e+09
19  2005    6.111893e+09
20  2007    5.216023e+09
21  2003    4.508044e+09
22  2000    3.843533e+09
23  2006    3.288324e+09
24  1991    2.021910e+09
25  1987    1.879386e+09
26  1994    1.810650e+09
27  1999    1.405354e+09
28  2001    3.991452e+08
29  1998    1.783399e+08
